In [ ]:
# Установка необходимых библиотек
!pip install fastapi uvicorn requests

# Импорт необходимых модулей
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict
import requests
import threading

# Создание экземпляра FastAPI
app = FastAPI()

# Модель данных для покупки
class Purchase(BaseModel):
    name: str
    group: str
    price: float
    unit: str
    quantity: int

# Хранилище данных о покупках
purchases: List[Purchase] = []

# Метод для добавления новой покупки
@app.post("/purchases/")
def add_purchase(purchase: Purchase):
    purchases.append(purchase)
    return {"message": "Purchase added successfully"}

# Метод для получения всех покупок
@app.get("/purchases/")
def get_purchases():
    return purchases

# Метод для получения расходов по каждой группе товаров и суммы всех покупок
@app.get("/purchases/summary/")
def get_purchase_summary():
    summary: Dict[str, float] = {}
    total_sum: float = 0.0

    for purchase in purchases:
        group = purchase.group
        cost = purchase.price * purchase.quantity

        if group in summary:
            summary[group] += cost
        else:
            summary[group] = cost

        total_sum += cost

    return {
        "summary": summary,
        "total_sum": total_sum
    }

# Запуск сервера FastAPI в отдельном потоке
def run_server():
    import uvicorn
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Запуск сервера в отдельном потоке
server_thread = threading.Thread(target=run_server)
server_thread.start()

# Тестирование API с использованием библиотеки requests
BASE_URL = "http://127.0.0.1:8000"

# Добавление покупок
purchases = [
    Purchase(name="Ноутбук", group="Электроника", price=1000.0, unit="шт", quantity=1),
    Purchase(name="Молоко", group="Продовольствие", price=2.5, unit="л", quantity=2),
    Purchase(name="Телефон", group="Электроника", price=500.0, unit="шт", quantity=1),
    Purchase(name="Хлеб", group="Продовольствие", price=1.0, unit="шт", quantity=3),
    Purchase(name="Книга", group="Книги", price=15.0, unit="шт", quantity=2),
    Purchase(name="Наушники", group="Электроника", price=50.0, unit="шт", quantity=1),
    Purchase(name="Яблоки", group="Продовольствие", price=1.5, unit="кг", quantity=4),
]

# Ограничим количество добавляемых покупок до 7
for i, purchase in enumerate(purchases):
    if i >= 7:
        break
    response = requests.post(f"{BASE_URL}/purchases/", json=purchase.dict())
    print("Добавление покупки:", response.json())

# Получение всех покупок
response = requests.get(f"{BASE_URL}/purchases/")
print("Все покупки:", response.json())

# Получение сводки по расходам
response = requests.get(f"{BASE_URL}/purchases/summary/")
print("Сводка по расходам:", response.json())

# Ожидание завершения сервера
server_thread.join()

INFO:     Started server process [8548]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:38498 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38500 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38510 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38518 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38528 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38540 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38550 - "POST /purchases/ HTTP/1.1" 200 OK
Добавление покупки: {'message': 'Purchase added successfully'}
INFO:     127.0.0.1:38556 - "GET /purchases/ HTTP/1.1" 200 OK
Все покупки: [{'name': 'Ноутбук', 'group': 'Электроника'